In [2]:
import requests
import json
import numpy as np
import pandas as pd
import math
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def indeedSearchUrl(pageInput,searchPeriod=0):
    if searchPeriod ==0:
        return 'https://sg.indeed.com/jobs?q=data+analyst&l=Singapore&radius=10&sort=date&start={0}0'.format(pageInput)
    else:
        return 'https://sg.indeed.com/jobs?q=data+analyst&l=Singapore&radius=10&sort=date&fromage={0}&start={1}0'.format(searchPeriod,pageInput)
print(indeedSearchUrl(0))

https://sg.indeed.com/jobs?q=data+analyst&l=Singapore&radius=10&sort=date&start=00


In [4]:
def getPages(indeedUrl):
    driver.get(indeedUrl)
    totalJobs = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'jobsearch-JobCountAndSortPane-jobCount'))
                ).text.replace(' jobs','').replace(',','')
    pages= math.ceil(int(totalJobs)/15)
    print(str(pages)+' pages')
    return pages

### Check JD for each job element

In [5]:
def scrapePage(indeedUrl):
    driver.get(indeedUrl)
    # Get Each Job Element 
    jobElements= WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'jobsearch-ResultsList'))
                ).find_elements(By.TAG_NAME, "h2")
    resultList = []
    for job in jobElements:
        # Get Job title
        title=job.text.replace('\n- job post', "").lower()
        titleFilter= ['intern','internship','senior','manager','lead','executive','director','research']
        if not any(substring in title for substring in titleFilter):     
            driver.implicitly_wait(10)
            # Get Link
            link=job.find_element(By.TAG_NAME, "a").get_attribute("href")
        
            # Open Job Details
            job.click()
            JD = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'jobsearch-ViewJobLayout-jobDisplay'))
                )
            # Check if from CareersFuture and skip
            jobSource=JD.find_element(By.CLASS_NAME, "jobsearch-JobMetadataFooter").text.replace('Report job','').strip()

            if jobSource!='MyCareersFuture.SG':
                JD_header= JD.find_element(By.CLASS_NAME, "jobsearch-JobComponent-embeddedHeader")
                JD_body= JD.find_element(By.CLASS_NAME, "jobsearch-JobComponent-embeddedBody")


                # Get Company
                company=JD_header.find_element(By.CSS_SELECTOR, "div[data-company-name='true']").text

                # Get Employment Type
                try:
                    salaryInfoAndJobType=JD_header.find_element(By.ID, "salaryInfoAndJobType")
                    try:
                        Etype = salaryInfoAndJobType.find_element(By.CLASS_NAME, "jobsearch-JobMetadataHeader-item").text
                    except:
                        Etype= 'N/A'
                    # Get Salary   
                    try:
                        salary= salaryInfoAndJobType.find_element(By.CLASS_NAME, "css-2iqe2o").text
                    except:
                        salary= 'N/A'
                except:
                    Etype= 'N/A'
                    salary= 'N/A'

                # Get Post Date
                postDate=JD_body.find_element(By.CLASS_NAME, "css-659xjq").text

                # Get Job Description
                jobDescriptionText=JD_body.find_element(By.ID, "jobDescriptionText").text  

                resultList.append([title,Etype,link,company,salary,postDate,jobDescriptionText])

    return resultList

In [6]:
driver = webdriver.Chrome('/Program Files (x86)/chromedriver')

In [9]:

searchPeriod=1
indeedUrl=indeedSearchUrl(0,searchPeriod)
print(indeedUrl)
pages=getPages(indeedUrl)
 
resultsList=[]
for i in range(pages):
    print(i)
    driver.implicitly_wait(10)
    resultsList.extend(scrapePage(indeedSearchUrl(i,searchPeriod)))
    

https://sg.indeed.com/jobs?q=data+analyst&l=Singapore&radius=10&sort=date&fromage=1&start=00
4 pages
0
1
2
3


In [10]:
resultNP=np.asarray(resultsList)

In [11]:
df = pd.DataFrame(resultNP, columns = ['title','Etype','link','company','salary','postDate','jobDescriptionText'])

In [12]:
df2=df[df['Etype'].str.contains('Full-time|Permanent')]
df2=df2[df2['postDate'] != 'Posted 30+ days ago']
df2=df2.drop_duplicates(subset=['title','company'], keep='last')

In [13]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 500)
df3=df2.reset_index(drop=True)
df3 = df3[['title','Etype','company','salary','postDate','jobDescriptionText','link']]
print(len(df3))
df3

8


,title,Etype,company,salary,postDate,jobDescriptionText,link
0,financial analyst (saas),- Permanent,Snaphunt Pte Ltd,"$2,000 - $4,000 a month",Posted today,"Company\nSnaphunt Pte Ltd\nsnaphunt.com\nDesignation\nFinancial Analyst (SaaS)\nDate Listed\n07 Mar 2023\nJob Type\nEntry Level / Junior Executive\nFull/Perm\nJob Period\nImmediate Start, Permanent\nProfession\nAccounting / Auditing / Taxation\nIndustry\nComputer and IT\nLocation Name\nSingapore\nAllowance / Remuneration\n$2,000 - 4,000 monthly\nCompany Profile\nSnaphunt is shaping how global employers hire in the new world of remote work. With Snaphunt, employers can easily build global teams by finding, engaging & hiring the best talent for their business- anywhere in the world.\nWe have over 14,000 employers and 3 million professionals on our platform and are backed by top venture capitalists. We operate via a fully remote/distributed international team and are looking to hire ambitious, independent problem solvers across multiple functions to support our business growth.\nJob Description\nBe part of a fully distributed international team\nJoin a fast growing startup that is sha...",https://sg.indeed.com/rc/clk?jk=d123f334db266492&fccid=d6545ed47a8e6a25&vjs=3
1,"sa/avp, investment services (data centre of excellence - data analyst)","Full-time, Permanent",Temasek International Pte Ltd,N/A,Posted Today,"The Investment Services Analytics Analyst will undertake research on relevant business topics/problems and conduct data driven analysis to draw out insights and synthesize findings with relevant stakeholders. In addition to the procured external datasets from our data vendors, there is a huge repository of data in the institutional databases that can be mined for insights. The Investment Services Analytics Analyst will engage various business functions in the Investment Services and Finance departments and provide pointers, where necessary, on data analytics with reference to business problems.\n\nResponsibilities :\nThe Investment Services Analyst will work alongside the business units in Investment Services and Finance departments to analyse and find analytical solutions to business questions.\nAs there are a wide range of data sets available in the systems stewarded by Investment Services and Finance business units, the Investment Services Analytics Analyst is expected to gain u...",https://sg.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bj-xBWVmzs7UIwFa-9Zm6MwNuQFkwLehssDYDaWnvIcTUgMX_84IeZ4HF_UABE9v6KoRUDg2vVc3nEHCdnJpD33mpeNobuGqywfPUDS1HbNLLC8bIrmWpaKMV_u373YMgL3KqnTiqPgaSCOvbZ7sqx8u6RBxjg_La-5EF_lZ0Wr73sxqVqxeGWVtltVnhTHBFuO61iQXwh_M48UZkE35oUbw9NKUaRpfqY4uwABIxbba7LwsXgt_lMMxLMZmh83sXk-YXIk4vv6jEIrJgoaRlKUil-DXkCOzZbPOmoVpQ_X7S4RMVDg39PDLokElWqlOEYxqVT29GXWoUdO55NnJVYhxBCJvSy8WzVY0cUXc4cc0dJ6x12vDUKQMphJhHO6akB9wU9RtBxgZ2R55nbEtEso2Q107u0JmiVNQurfqVrATZEYvCRawdnTMeXIYdhUuOAEXDlvvX4Pg7IDzvCI8oc4TqT1qrI-16qEsHl-EgXPQgbJt5HDsGcbLoZI0aqRX0ahfA1B4EsDqZ_pyW9sUYctm1K7hJEWJVEvqJAKfvfZNbsoDlsgz_pTepJ2nd-1xt8imZT0_TwgxwY7eGZ22gdnd5BRdWMdVs_0Xc6Eix0sIR3dxqjD6AlGK-U7n3EuhGIhwK7wCYQSsdX119OXkWmj1Q3X1tM00ZymzmcPS_fUQidFiihePWVFx-awBGkBT4xamcd8r9cdwysC4NtuR30ASrPN1ORVeanU3eqgn9ir1vYzIHHPAQ6KNf7-DF5fGUs0YpYJIwjzN9CpfCulqCtgk0koDODsrce_qPQXSBE6FHeb9Q4jzp_AFbLrivtE4msOrYniei2PgOFrQpjdMwo8-y40fIL0GrS8IjpmZq3uw==&xkcb=SoA6-_M3SdeMAn2Xtp0KbzkdCdPP&p=3&fvj=0&vjs=3
2,business analyst ( product/sales/data),- Permanent,Volt,"$5,000 - $6,000 a month",Posted Today,"Location:\nSingapore\n\nJob Type:\nPermanent\n\nSalary:\nS$5000 - S$6000 per month\n\nReference:\nBBBH10708_1678085338\n\nContact:\nDarren Ou\n\nEmail:\nemail Darren\n\nDarren Ou\nEmail Darren\nBusiness Analyst\n\nYou will be responsible to support multiple business units and be a point of liaison, bridging the requirements between business units and clients.\n\nKey Accountabilities:\n\n\n\n\n\n\nSupport in developing solutions to clients by applying the broad understanding of web/mobile technologies and SaaS platforms to solve real-